In [1]:
!pip install pgmpy

In [2]:
!pip install matplotlib

In [3]:
import pandas as pd
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import MmhcEstimator
from pgmpy.estimators import K2Score
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator

In [4]:
import networkx as nx
import matplotlib.pyplot as plt
from pgmpy.readwrite import BIFReader, BIFWriter

In [5]:
characteristics_df = pd.read_csv('data/characteristics_prepped.csv', encoding='latin1')
places_df = pd.read_csv('data/places_prepped.csv', encoding='latin1')
users_df = pd.read_csv('data/users_prepped.csv', encoding='latin1')
vehicles_df = pd.read_csv('data/vehicles_prepped.csv', encoding='latin1')

## Vehicles + users merge

Vechicles and users table might contain multiple rows for the same Num_Acc id. (Because there might be multiple people or vehicles involved in the accident)

In [6]:
vehicles_df.shape

(1433389, 8)

In [7]:
users_df.shape

(1876005, 13)

In [8]:
vehicles_users_df = pd.merge(vehicles_df, users_df, on='Num_Acc', how='outer', suffixes=('_vehicles', '_users'))

Filter rows that that contains a user-vehicle pair that does not match.

In [9]:
vehicles_users_df = vehicles_users_df[vehicles_users_df['num_veh_vehicles'] == vehicles_users_df['num_veh_users']]

In [10]:
vehicles_users_df.head()

,Num_Acc,Occupants,Mobile Obstacle,Shock,num_veh_vehicles,Vehicle_Category,Fix_obstacle_type,Maneuver_category,Seat,User type,Severity,Sex,Trip,Pedestrian location,Pedestrian action,Pedestrian alone,num_veh_users,Age_range,Safety equipment existence,Safety equipment use
0,200500000001,1,2.0,1.0,A01,8,0,1,1,1,2,1,1,0,0,0,A01,3,1,1
7,200500000001,1,2.0,8.0,B02,8,0,1,1,1,3,2,3,0,0,0,B02,3,1,1
8,200500000001,1,2.0,8.0,B02,8,0,1,2,2,1,1,0,0,0,0,B02,3,1,1
9,200500000001,1,2.0,8.0,B02,8,0,1,4,2,1,1,0,0,0,0,B02,0,3,1
10,200500000001,1,2.0,8.0,B02,8,0,1,5,2,1,1,0,0,0,0,B02,0,1,1


In [11]:
vehicles_users_df['num_veh_vehicles'].isna().sum()

0

In [12]:
vehicles_users_df['num_veh_users'].isna().sum()

0

In [13]:
vehicles_users_df.drop(['num_veh_users', 'num_veh_vehicles'], axis=1, inplace=True)

In [14]:
vehicles_users_df.shape

(1875983, 18)

There are no vehicles without users and no users without vehicles.

## Characteristics + places merge

In [15]:
places_df.shape

(839985, 11)

In [16]:
characteristics_df.shape

(669966, 11)

In [17]:
characteristics_places_df = pd.merge(places_df, characteristics_df, on='Num_Acc', how='inner')

In [18]:
characteristics_places_df.shape

(669966, 21)

## Merge all

In [19]:
merged_df = pd.merge(characteristics_places_df, vehicles_users_df, on='Num_Acc', how='inner')

In [20]:
merged_df.shape

(1491407, 38)

In [21]:
merged_df.head()

,Num_Acc,road_type,traffic_type,num_lanes,bike_lane_type,terrain_type,curve_type,condition,infrastructure,situ,...,User type,Severity,Sex,Trip,Pedestrian location,Pedestrian action,Pedestrian alone,Age_range,Safety equipment existence,Safety equipment use
0,201600000001,3.0,2.0,0.0,0.0,1.0,3.0,1.0,0.0,1.0,...,1,1,2,0,0,0,0,3,1,1
1,201600000001,3.0,2.0,0.0,0.0,1.0,3.0,1.0,0.0,1.0,...,1,3,1,9,0,0,0,1,2,1
2,201600000002,3.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,...,1,3,1,5,0,0,0,4,1,1
3,201600000002,3.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,...,2,3,1,0,0,0,0,1,1,1
4,201600000002,3.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,...,2,3,2,0,0,0,0,4,1,1


In [22]:
merged_df.drop('Num_Acc', axis=1, inplace=True)
merged_df.drop('Municipality', axis=1, inplace=True)
merged_df.drop('Department', axis=1, inplace=True)
merged_df.drop('Date', axis=1, inplace=True)

In [23]:
merged_df = merged_df.apply(pd.to_numeric, downcast="integer")

## Bayes model

### Training the model

In [24]:
scoring_method = K2Score(data=merged_df)
est = HillClimbSearch(data=merged_df)
estimated_model = est.estimate(
    scoring_method=scoring_method, max_indegree=4, max_iter=1000
)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [25]:
model = BayesianNetwork(estimated_model)

In [26]:
model.fit(merged_df, estimator=BayesianEstimator)

In [27]:
model.save("model_indegree4.xml", filetype="xmlbif")

In [28]:
nx.nx_agraph.to_agraph(model).draw("graph.png", prog="dot")

### Sampling from the model (Variable elimination)

#### 1. Severity without evidences
If a person is involved in an accident, how severe will their injuries be.

Severity meaning

*   1 - Unscathed/Unharmed
*   2 - Light injury
*   3 - Hospitalized wounded
*   4 - Killedled

In [30]:
from pgmpy.inference import VariableElimination
inference = VariableElimination(model)

evidence_set = {}

query_result = inference.query(variables=["Severity"],
                               evidence=evidence_set)
print(query_result)

+-------------+-----------------+
| Severity    |   phi(Severity) |
+=============+=================+
| Severity(1) |          0.4088 |
+-------------+-----------------+
| Severity(2) |          0.3562 |
+-------------+-----------------+
| Severity(3) |          0.2079 |
+-------------+-----------------+
| Severity(4) |          0.0270 |
+-------------+-----------------+


Results:

* It is most likely they will be unharmed (41%)
* A high chance of light injury (36%)
* A relatively high chance of being hospitalized (21%)
* And a low chance of death (3%)

> The sum is above 100% because of rounding

#### 2. Cycling in Île-de-France without safety equipment

##### 1.1 If a cyclist is in an accident with no safety equipment in Île-de-France region, what is the chance of them dying.

In [30]:
evidence_set = {"Regions": 10, # Île-de-France
                "Vehicle_Category": 1, # Bicycle
                "Safety equipment use": 2 # No safety equipment
               }

query_result = inference.query(variables=["Severity"],
                               evidence=evidence_set)
print(query_result)

+-------------+-----------------+
| Severity    |   phi(Severity) |
+=============+=================+
| Severity(1) |          0.0996 |
+-------------+-----------------+
| Severity(2) |          0.5020 |
+-------------+-----------------+
| Severity(3) |          0.3604 |
+-------------+-----------------+
| Severity(4) |          0.0380 |
+-------------+-----------------+


##### Result:
They have a **3.8%** chance of dying 

##### 1.2 What if they do wear safety equipment with these conditions?

In [31]:
evidence_set = {"Regions": 10, # Île-de-France
                "Vehicle_Category": 1, # Bicycle
                "Safety equipment use": 1 # Wearing safety equipment
               }

query_result = inference.query(variables=["Severity"],
                               evidence=evidence_set)
print(query_result)

+-------------+-----------------+
| Severity    |   phi(Severity) |
+=============+=================+
| Severity(1) |          0.1168 |
+-------------+-----------------+
| Severity(2) |          0.4713 |
+-------------+-----------------+
| Severity(3) |          0.3800 |
+-------------+-----------------+
| Severity(4) |          0.0319 |
+-------------+-----------------+


##### Result:
They have a lower (**3.2%**) chance of dying. (Compared to 3.8%) There is more probability of being hospitalized this way (2%) but I'm not sure why. The chance of being unharmed is also significantly higher.

##### 1.3 How about other regions?

In [32]:
evidence_set = {"Severity": 4, # Dying
                "Vehicle_Category": 1, # Bicycle
                "Safety equipment use": 1 # Wearing safety equipment
               }

query_result = inference.query(variables=["Regions"],
                               evidence=evidence_set)
print(query_result)

evidence_set = {"Severity": 4, # Dying
                "Vehicle_Category": 1, # Bicycle
                "Safety equipment use": 2 # No safety equipment
               }

query_result = inference.query(variables=["Regions"],
                               evidence=evidence_set)
print(query_result)

+-------------+----------------+
| Regions     |   phi(Regions) |
+=============+================+
| Regions(1)  |         0.1114 |
+-------------+----------------+
| Regions(2)  |         0.0374 |
+-------------+----------------+
| Regions(3)  |         0.0449 |
+-------------+----------------+
| Regions(4)  |         0.0391 |
+-------------+----------------+
| Regions(5)  |         0.0093 |
+-------------+----------------+
| Regions(6)  |         0.0070 |
+-------------+----------------+
| Regions(7)  |         0.0657 |
+-------------+----------------+
| Regions(8)  |         0.0089 |
+-------------+----------------+
| Regions(9)  |         0.0628 |
+-------------+----------------+
| Regions(10) |         0.2049 |
+-------------+----------------+
| Regions(11) |         0.0081 |
+-------------+----------------+
| Regions(12) |         0.0009 |
+-------------+----------------+
| Regions(13) |         0.0475 |
+-------------+----------------+
| Regions(14) |         0.0920 |
+---------

If someone dies in an accident while biking (with or without safety equipment) it is most likely they were in **Île-de-France** (10) region.

While this might be shocking, let's look at some other information first. 

##### 1.4 *If we are riding a bike, and take part in an accident, how likely it is that we are in Île-de-France?*

In [33]:
evidence_set = {
                "Vehicle_Category": 1 # Bicycle
               }

query_result = inference.query(variables=["Regions"],
                               evidence=evidence_set)
print(query_result)

+-------------+----------------+
| Regions     |   phi(Regions) |
+=============+================+
| Regions(1)  |         0.0967 |
+-------------+----------------+
| Regions(2)  |         0.0283 |
+-------------+----------------+
| Regions(3)  |         0.0396 |
+-------------+----------------+
| Regions(4)  |         0.0329 |
+-------------+----------------+
| Regions(5)  |         0.0072 |
+-------------+----------------+
| Regions(6)  |         0.0067 |
+-------------+----------------+
| Regions(7)  |         0.0563 |
+-------------+----------------+
| Regions(8)  |         0.0055 |
+-------------+----------------+
| Regions(9)  |         0.0595 |
+-------------+----------------+
| Regions(10) |         0.2828 |
+-------------+----------------+
| Regions(11) |         0.0049 |
+-------------+----------------+
| Regions(12) |         0.0006 |
+-------------+----------------+
| Regions(13) |         0.0327 |
+-------------+----------------+
| Regions(14) |         0.0869 |
+---------

As we can see here, **bike accidents in general are more common in Île-de-France.**

This might be because Île-de-France is a more dangerous place for bikers **or** what is more likely, **people in Île-de-France are more likely to ride bikes!**

With some quick google searches, **Paris** (which is in Île-de-France region) is the **second most bike-friendly city** in France.

Also, from the first **Tour de France** in **1903** to **1939**, the **Grand Depart** (the start of the race) was always in the **Île-de-France region**. Because of this, it is very likely that there is a deep culture of cycling there and a well-built infrastructure for it!

> Sources:
> https://www.canyon.com/en-cn/blog-content/hybrid-bike-news/bike-friendly-cities-france/b02042022.html
> https://en.wikipedia.org/wiki/List_of_Tour_de_France_Grands_D%C3%A9parts

##### 1.5 Is it more dangerous if we get into an accident at night?

In [34]:
evidence_set = {"Vehicle_Category": 1, # Bicycle
                "Safety equipment use": 2, # No safety equipment
                "Lighting": 5 # Night without public lighting
               }

query_result = inference.query(variables=["Severity"],
                               evidence=evidence_set)

print("Night")
print(query_result)

evidence_set = {"Vehicle_Category": 1, # Bicycle
                "Safety equipment use": 2, # No safety equipment
                "Lighting": 1 # Full day
               }

query_result = inference.query(variables=["Severity"],
                               evidence=evidence_set)
print("Day")
print(query_result)

Night
+-------------+-----------------+
| Severity    |   phi(Severity) |
+=============+=================+
| Severity(1) |          0.1195 |
+-------------+-----------------+
| Severity(2) |          0.3869 |
+-------------+-----------------+
| Severity(3) |          0.4170 |
+-------------+-----------------+
| Severity(4) |          0.0765 |
+-------------+-----------------+
Day
+-------------+-----------------+
| Severity    |   phi(Severity) |
+=============+=================+
| Severity(1) |          0.1047 |
+-------------+-----------------+
| Severity(2) |          0.4829 |
+-------------+-----------------+
| Severity(3) |          0.3693 |
+-------------+-----------------+
| Severity(4) |          0.0432 |
+-------------+-----------------+


##### Result:

The chance of the cyclist dying in an accident is much higher at night, nearly twice as much as in proper lighting conditions. The cyclists with no injuries are nearly the same percentage. The other (2 and 3) severities show that during the day, cyclists are more likely to get lightly injured and not hospitalized, however at night it is more probable that we get a more serious injury.

> In conclusion, it is always recommended it be cautious when traveling at night!

Another factor in this statistic that we have no information on from our dataset is whether these cyclists were wearing proper visibility equipment at the time of these accidents. 

##### 1.6 What about safety equipment at night?

In [35]:
evidence_set = {"Vehicle_Category": 1, # Bicycle
                "Safety equipment use": 1, # Wearing safety equipment
                "Lighting": 5 # Night without public lighting
               }

query_result = inference.query(variables=["Severity"],
                               evidence=evidence_set)
print(query_result)

+-------------+-----------------+
| Severity    |   phi(Severity) |
+=============+=================+
| Severity(1) |          0.1157 |
+-------------+-----------------+
| Severity(2) |          0.3685 |
+-------------+-----------------+
| Severity(3) |          0.4561 |
+-------------+-----------------+
| Severity(4) |          0.0596 |
+-------------+-----------------+


##### Result:

Instead of **7.7%** of dying, we have a **6%** chance this way, so safety equipment is a win even in nightly conditions. The chance of being hospitalized is higher here by 4% than without having safety equipment, which is a bit odd. 

We sadly have no information on whether safety equipment here also refers to visibility equipment and lamps, which are necessary for safe biking in these lighting conditions.

#### 3. How old are motorbike riders?

##### 3.1 What does it tell about our age, if we are riding a motorcycle and get into an accident?

In [36]:
evidence_set = {"Vehicle_Category": 4 # Motorcycle
               }

query_result = inference.query(variables=["Age_range"],
                               evidence=evidence_set)
print(query_result)

+--------------+------------------+
| Age_range    |   phi(Age_range) |
+==============+==================+
| Age_range(0) |           0.0088 |
+--------------+------------------+
| Age_range(1) |           0.0147 |
+--------------+------------------+
| Age_range(2) |           0.2360 |
+--------------+------------------+
| Age_range(3) |           0.4837 |
+--------------+------------------+
| Age_range(4) |           0.2385 |
+--------------+------------------+
| Age_range(5) |           0.0184 |
+--------------+------------------+


##### Result:

As we can see, if we do have an accident on a motorcycle, there is a 50 percent chance that we are in the 25-45 age group!

#### 4. Gender and severity

##### 4.1 Driving in the city on well lit days, how severe are the injuries of men vs women in an accident?

In [37]:
evidence_set = {"Localisation": 2, # Built in areas
                "User type": 1, # Driver
                "Lighting": 1, # Full day
                "Vehicle_Category": 8, # In a car
                "Sex": 1 # Men
               }

query_result = inference.query(variables=["Severity"],
                               evidence=evidence_set)
print("Men")
print(query_result)

evidence_set = {"Localisation": 2, # Built in areas
                "User type": 1, # Driver
                "Lighting": 1, # Full day
                "Vehicle_Category": 8, # In a car
                "Sex": 2 # Women
               }

query_result = inference.query(variables=["Severity"],
                               evidence=evidence_set)
print("Women")
print(query_result)

Men
+-------------+-----------------+
| Severity    |   phi(Severity) |
+=============+=================+
| Severity(1) |          0.6988 |
+-------------+-----------------+
| Severity(2) |          0.2017 |
+-------------+-----------------+
| Severity(3) |          0.0864 |
+-------------+-----------------+
| Severity(4) |          0.0131 |
+-------------+-----------------+
Women
+-------------+-----------------+
| Severity    |   phi(Severity) |
+=============+=================+
| Severity(1) |          0.5658 |
+-------------+-----------------+
| Severity(2) |          0.3305 |
+-------------+-----------------+
| Severity(3) |          0.0962 |
+-------------+-----------------+
| Severity(4) |          0.0075 |
+-------------+-----------------+


##### Result:

This shows that as driving a vehicle in daylight in a city, men are much more likely to have no injuries (**70% vs 56.5%**) than women. Women are more likely to have light injuries (**men 20%, women 33%**) and a bit more likely to be hospitalized (**men 8.6%, women 9.6%**). In these conditions, men are more likely to die (**men 1.3%, women 0.7%**).

There are a few different factors that might play into these differences. Maybe when encountering an accident, women are more prone to get injured because of biological differences. It might be that some male drivers downplay their injuriesor maybe male drivers take part in more minor accidents than women (in these conditions).

##### 4.2 How about in rural areas?

In [38]:
evidence_set = {"Localisation": 1, # Out of agglomeration
                "User type": 1, # Driver
                "Lighting": 1, # Full day
                "Vehicle_Category": 8, # In a car
                "Sex": 1 # Men
               }

query_result = inference.query(variables=["Severity"],
                               evidence=evidence_set)
print("Men")
print(query_result)

evidence_set = {"Localisation": 1, # Out of agglomeration
                "User type": 1, # Driver
                "Lighting": 1, # Full day
                "Vehicle_Category": 8, # In a car
                "Sex": 2 # Women
               }

query_result = inference.query(variables=["Severity"],
                               evidence=evidence_set)
print("Women")
print(query_result)

Men
+-------------+-----------------+
| Severity    |   phi(Severity) |
+=============+=================+
| Severity(1) |          0.5260 |
+-------------+-----------------+
| Severity(2) |          0.2426 |
+-------------+-----------------+
| Severity(3) |          0.1921 |
+-------------+-----------------+
| Severity(4) |          0.0393 |
+-------------+-----------------+
Women
+-------------+-----------------+
| Severity    |   phi(Severity) |
+=============+=================+
| Severity(1) |          0.4022 |
+-------------+-----------------+
| Severity(2) |          0.3750 |
+-------------+-----------------+
| Severity(3) |          0.2017 |
+-------------+-----------------+
| Severity(4) |          0.0212 |
+-------------+-----------------+


##### Result:

The severity of injury is the same trend as before for men vs women. The difference now is that in these conditions, people are more likely to get injured in not built in areas!

##### 4.3 Is there a correlation between gender and user type?

User types:

*   1 - Driver
*   2 - Passenger
*   3 - Pedestrian
*   4 - Pedestrian in rollerblade or scooter

In [39]:
evidence_set = {"Localisation": 2, # Built in areas
                "Lighting": 1, # Full day
                "Vehicle_Category": 8, # In a car
                "Sex": 1 # Men
               }

query_result = inference.query(variables=["User type"],
                               evidence=evidence_set)
print("Men")
print(query_result)

evidence_set = {"Localisation": 2, # Built in areas
                "Vehicle_Category": 8, # In a car
                "Lighting": 1, # Full day
                "Sex": 2 # Women
               }

query_result = inference.query(variables=["User type"],
                               evidence=evidence_set)
print("Women")
print(query_result)

Men
+--------------+------------------+
| User type    |   phi(User type) |
+==============+==================+
| User type(1) |           0.7594 |
+--------------+------------------+
| User type(2) |           0.1318 |
+--------------+------------------+
| User type(3) |           0.1059 |
+--------------+------------------+
| User type(4) |           0.0029 |
+--------------+------------------+
Women
+--------------+------------------+
| User type    |   phi(User type) |
+==============+==================+
| User type(1) |           0.5988 |
+--------------+------------------+
| User type(2) |           0.2123 |
+--------------+------------------+
| User type(3) |           0.1862 |
+--------------+------------------+
| User type(4) |           0.0027 |
+--------------+------------------+


##### Result:

Men are more likely to be drivers in these conditions (**men 76%, women 60%**) and women are more likely to be passengers or pedestrians than men.

If we can assume that probably same amount of man and woman pedestrians get in an accident, then from this I would assume that there are more male drivers, so they are more likely to be drivers in an accident than pedestrians.



#### 5. Age and severity

The ages are within these ranges:

[0, 12, 16, 25, 45, 65, 150]

##### 5.1 What age groups are more likely to get hospitalized because of an accident? (assuming they are the driver)

In [104]:
evidence_set = {"User type": 1, # Driver
                "Vehicle_Category": 8, # In a car
                "Severity": 3 # Hospitalized
               }

query_result = inference.query(variables=["Age_range"],
                               evidence=evidence_set)
print(query_result)

+--------------+------------------+
| Age_range    |   phi(Age_range) |
+==============+==================+
| Age_range(0) |           0.0003 |
+--------------+------------------+
| Age_range(1) |           0.0007 |
+--------------+------------------+
| Age_range(2) |           0.2113 |
+--------------+------------------+
| Age_range(3) |           0.4327 |
+--------------+------------------+
| Age_range(4) |           0.2651 |
+--------------+------------------+
| Age_range(5) |           0.0899 |
+--------------+------------------+


##### Result:

It is more likely to have a driver in a car to be between 25 and 45 years old. It is important to notice that the age range 2 is only 9 years, yet there is a 21 percent chance that this is the age of the driver.

##### 5.2 Which age groups are more likely to use safety equipments?

In [105]:
evidence_set = {"User type": 1, # Driver
                "Safety equipment use": 1, # Wearing safety equipment
               }

query_result = inference.query(variables=["Age_range"],
                               evidence=evidence_set)
print(query_result)

+--------------+------------------+
| Age_range    |   phi(Age_range) |
+==============+==================+
| Age_range(0) |           0.0015 |
+--------------+------------------+
| Age_range(1) |           0.0198 |
+--------------+------------------+
| Age_range(2) |           0.2368 |
+--------------+------------------+
| Age_range(3) |           0.4267 |
+--------------+------------------+
| Age_range(4) |           0.2459 |
+--------------+------------------+
| Age_range(5) |           0.0693 |
+--------------+------------------+


##### Result:

These results probably correlate with how age groups are represented in an accident (most people are probably between 25 and 45. Then a lot between 45 and 65, and between 16-25)

#### 6. Analysing trip type

What are the likely reasons a driver 25-45 years old vs 16-25 started a trip that lead to hospitalization when driving during the day?

Trip types

*   1 - Home - Work
*   2 - Home - School
*   3 - Shopping
*   4 - Professional use
*   5 - Walking- Leisure
*   9 - Other
*   0 - no information

In [31]:
evidence_set = {"User type": 1, # Driver
                "Age_range": 3, # 25-45
                "Severity": 3, # Hospitalized
                "Lighting": 1 # Full day
               }

query_result = inference.query(variables=["Trip"],
                               evidence=evidence_set)
print("25-45 years old")
print(query_result)

evidence_set = {"User type": 1, # Driver
                "Age_range": 2, # 16-25
                "Severity": 3, # Hospitalized
                "Lighting": 1 # Full day
               }

query_result = inference.query(variables=["Trip"],
                               evidence=evidence_set)
print("16-25 years old")
print(query_result)

25-45 years old
+---------+-------------+
| Trip    |   phi(Trip) |
+=========+=============+
| Trip(0) |      0.2195 |
+---------+-------------+
| Trip(1) |      0.2342 |
+---------+-------------+
| Trip(2) |      0.0073 |
+---------+-------------+
| Trip(3) |      0.0186 |
+---------+-------------+
| Trip(4) |      0.1449 |
+---------+-------------+
| Trip(5) |      0.3073 |
+---------+-------------+
| Trip(9) |      0.0683 |
+---------+-------------+
16-25 years old
+---------+-------------+
| Trip    |   phi(Trip) |
+=========+=============+
| Trip(0) |      0.2235 |
+---------+-------------+
| Trip(1) |      0.1700 |
+---------+-------------+
| Trip(2) |      0.0486 |
+---------+-------------+
| Trip(3) |      0.0148 |
+---------+-------------+
| Trip(4) |      0.0702 |
+---------+-------------+
| Trip(5) |      0.3995 |
+---------+-------------+
| Trip(9) |      0.0734 |
+---------+-------------+


Result:

Among both groups, leisure trips are the most common. Naturally, home-work trips are more common among 25-45 year olds, and home-school trips are more common among 16-25 year olds. Professional use is understandably more common in the first group.

#### 7. What are the most dangerous road conditions?

What road conditions are likely when a middle-aged, likely experienced driver suffers a deadly accident?

In [33]:
evidence_set = {"User type": 1, # Driver
                "Age_range": 3, # 25-45
                "Severity": 4, # Killed
               }

query_result = inference.query(variables=["condition"],
                               evidence=evidence_set)

print("Road conditions:")
print(query_result)

query_result = inference.query(variables=["terrain_type"],
                               evidence=evidence_set)

print("Terrain type:")
print(query_result)

query_result = inference.query(variables=["curve_type"],
                               evidence=evidence_set)

print("Curve type:")
print(query_result)

Road conditions:
+--------------+------------------+
| condition    |   phi(condition) |
+==============+==================+
| condition(0) |           0.0306 |
+--------------+------------------+
| condition(1) |           0.7802 |
+--------------+------------------+
| condition(2) |           0.1696 |
+--------------+------------------+
| condition(3) |           0.0011 |
+--------------+------------------+
| condition(4) |           0.0004 |
+--------------+------------------+
| condition(5) |           0.0029 |
+--------------+------------------+
| condition(6) |           0.0006 |
+--------------+------------------+
| condition(7) |           0.0063 |
+--------------+------------------+
| condition(8) |           0.0029 |
+--------------+------------------+
| condition(9) |           0.0054 |
+--------------+------------------+
Terrain type:
+-----------------+---------------------+
| terrain_type    |   phi(terrain_type) |
+=================+=====================+
| terrain_type(

Result:
If a driver aged between 25-45 dies, the road had normal conditions with **78%** probability and was wet with **17%** probability.
Similarly **70%** flat terrain and **16%** slope, hills are less common.
Also similarly **70%** probability of having a straight road and **22%** probability of being curved in either direction.

These could be due to the general distribution of all driving situations.

#### 8. Are accidents likely to be worse during holidays?

Are drivers likely to injured worse during the day on a holiday?

In [34]:
evidence_set = {"User type": 1, # Driver
                "Lighting": 1, # Full day
                "Holiday": 0 # No holiday
               }

query_result = inference.query(variables=["Severity"],
                               evidence=evidence_set)

print("No holiday:")
print(query_result)

evidence_set = {"User type": 1, # Driver
                "Lighting": 1, # Full day
                "Holiday": 1 # Holiday
               }

query_result = inference.query(variables=["Severity"],
                               evidence=evidence_set)

print("Holiday:")
print(query_result)

No holiday:
+-------------+-----------------+
| Severity    |   phi(Severity) |
+=============+=================+
| Severity(1) |          0.4515 |
+-------------+-----------------+
| Severity(2) |          0.3381 |
+-------------+-----------------+
| Severity(3) |          0.1874 |
+-------------+-----------------+
| Severity(4) |          0.0229 |
+-------------+-----------------+
Holiday:
+-------------+-----------------+
| Severity    |   phi(Severity) |
+=============+=================+
| Severity(1) |          0.4491 |
+-------------+-----------------+
| Severity(2) |          0.3393 |
+-------------+-----------------+
| Severity(3) |          0.1886 |
+-------------+-----------------+
| Severity(4) |          0.0231 |
+-------------+-----------------+


Result:

There is no significant difference in the probabilities regarding the severity of the accident, whether it was a holiday or not.

### Gibbs Sampling

In [58]:
model_l = BayesianNetwork.load("model_indegree4.xml", filetype="xmlbif")

In [70]:
gibbs_chain.cardinalities

{'intel': 2, 'sat': 2}

In [ ]:
from pgmpy.sampling import GibbsSampling

gibbs_chain = GibbsSampling(model_l)

In [71]:
gibbs_chain.sample(size=1)

0it [00:00, ?it/s]

,intel,sat
0,0,0
